In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_path: Path
    C: float
    penalty: str
    solver: str
    class_weight: str  # use Optional[str] if you plan to set None
    max_iter: int
    random_state: int

In [4]:
from hr_attrition.constants import *
from hr_attrition.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


   

    def get_model_trainer_config(self) -> ModelTrainingConfig:
     config = self.config.model_training
     params = self.params.model_params

     create_directories([config.root_dir])

     model_trainer_config = ModelTrainingConfig(
        root_dir=Path(config.root_dir),
        train_data_path=Path(config.train_data_path),
        test_data_path=Path(config.test_data_path),
        model_path=Path(config.model_path),
        C=params.C,
        penalty=params.penalty,
        solver=params.solver,
        class_weight=params.class_weight,
        max_iter=params.max_iter,
        random_state=params.random_state
    )

     return model_trainer_config

    


    

In [8]:
import pandas as pd
import os
from hr_attrition.utils.logging import logger
from sklearn.linear_model import LogisticRegression
import joblib


class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def train(self):
        # Load the data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target
        train_x = train_data.drop("Attrition", axis=1)
        test_x = test_data.drop("Attrition", axis=1)
        train_y = train_data["Attrition"]
        test_y = test_data["Attrition"]

        # Initialize and train logistic regression model
        model = LogisticRegression(
            C=self.config.C,
            penalty=self.config.penalty,
            solver=self.config.solver,
            class_weight=self.config.class_weight,
            max_iter=self.config.max_iter,
            random_state=self.config.random_state
        )
        model.fit(train_x, train_y)

        # Save model to disk
        joblib.dump(model, self.config.model_path)

        logger.info(f"Model trained and saved at {self.config.model_path}")


In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e


[2025-04-16 16:03:52,130: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-16 16:03:52,134: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-16 16:03:52,140: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-16 16:03:52,141: INFO: common: created directory at: artifacts]
[2025-04-16 16:03:52,143: INFO: common: created directory at: artifacts/model_training]
[2025-04-16 16:03:52,618: INFO: 3934808436: Model trained and saved at artifacts\model_training\logreg_model.pkl]
